In [158]:
import numpy as np
import pandas as pd  
import time

In [159]:
dfUser = pd.read_csv('../data_csv/User.csv')
dfOrderDetail = pd.read_csv('../data_csv/OrderDetail.csv')
dfOrderDetail = dfOrderDetail.rename(columns= {"code" : 'code_order' , 'name' : 'name_order'})
dfCustomerDetail = pd.read_csv('../data_csv/CustomerDetail.csv')
dfCustomerDetail = dfCustomerDetail.drop(columns= {'id' , 'creator_id' , 'created_at' , 'updated_at' , 'deleted_at' , 'email' , 'phone' , 'fax' , 'tax_number' , 'address' , 'media_id' , 'user_id'})
dfCustomerDetail = dfCustomerDetail.rename(columns= {'code' : 'code_cus' , 'name' : 'name_cus' })
# dfCustomerDetail.head(2)
dfOrderDetail = dfOrderDetail.merge(dfCustomerDetail , on = 'customer_id' , how = 'left')
dfOrderDetail.loc[0]


code_order                                  DYC-US-220425-Ginè Energy-558
name_order                                  DYC-US-220425-Ginè Energy-558
qrcode                                                     1666975066DH32
description             <p>Đơn h&agrave;ng Reorder DYC-US-220219-Gin&e...
quantity                                                            100.0
creator_id                                                            234
created_at                                      2022-05-05 17:05:01+00:00
updated_at                                      2022-10-28 23:37:47+00:00
deleted_at                                                            NaN
color                                                                 NaN
size                                                                  NaN
po                                                                    NaN
tolerance                                                           103.0
loss_percentage                       

In [160]:
dfProductionTracking = pd.read_csv('../data_csv/ProductionTracking.csv')
dfProductionTrackingDetail = pd.read_csv('../data_csv/ProductionTrackingDetail.csv')
dfManufactureDeptDetail = pd.read_csv('../data_csv/ManufactureDeptDetail.csv')
dfProductionNotExistOrder = pd.read_csv('../data_csv/ProductionNotExistOrder.csv')

dfManufactureDeptDetail = dfManufactureDeptDetail.rename(columns= {'code' : 'code_dept' , 'name' : 'name_dept'})
dfProductionNotExistOrder = dfProductionNotExistOrder.rename(columns= {'id' : 'prod_not_exist_order_id','created_at' : 'not_exist_create' , 'name' : 'name_not_exist'}).drop(columns= {'updated_at' , 'deleted_at' , 'qrcode'})

In [161]:
dfProductionTracking = dfProductionTracking.merge(dfOrderDetail[['code_order' , 'name_order' , 'order_id' , 'name_cus' , 'quantity']], on = 'order_id' , how = 'left')
dfProductionTracking = dfProductionTracking.merge(dfManufactureDeptDetail[['code_dept' , 'name_dept' , 'manufacture_dept_id']], on = 'manufacture_dept_id' , how = 'left')
dfProductionTracking = dfProductionTracking.merge(dfProductionNotExistOrder, on = 'prod_not_exist_order_id' , how = 'left')

In [162]:
dfProductionTracking.head(2)

,id,created_at,updated_at,deleted_at,order_id,prod_not_exist_order_id,manufacture_dept_id,target_quantity,actual_quantity,complete_percent,code_order,name_order,name_cus,quantity,code_dept,name_dept,name_not_exist,not_exist_create,buyer_name,style_name
0,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,NaN,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet
1,2,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,2.0,8,301.0,0.0,0.0,NaN,NaN,NaN,NaN,LEAN 4,May 4,PO38046SAND,2022-10-22 03:00:00+00:00,SA,PO 38046 Charter Hat Sand


In [163]:
dfProductionTrackingFix = dfProductionTracking.drop(columns= {'id','updated_at', 'deleted_at' , 'order_id' , 'manufacture_dept_id', 'order_id', 'prod_not_exist_order_id' })
dfProductionTrackingFix

,created_at,target_quantity,actual_quantity,complete_percent,code_order,name_order,name_cus,quantity,code_dept,name_dept,name_not_exist,not_exist_create,buyer_name,style_name
0,2022-10-22 03:00:00+00:00,927.0,318.0,34.30,NaN,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet
1,2022-10-22 03:00:00+00:00,301.0,0.0,0.00,NaN,NaN,NaN,NaN,LEAN 4,May 4,PO38046SAND,2022-10-22 03:00:00+00:00,SA,PO 38046 Charter Hat Sand
2,2022-10-22 03:00:00+00:00,268.0,2.0,0.75,NaN,NaN,NaN,NaN,LEAN 5,May 5,PO38050ROL_TAN,2022-10-22 03:00:00+00:00,SA,PO38050 Kids Clear Creek Boonie
3,2022-10-22 03:00:00+00:00,289.0,0.0,0.00,NaN,NaN,NaN,NaN,LEAN 6,May 6,PO38055_ULTR_PUMI,2022-10-22 03:00:00+00:00,SA,PO38056_Ultra adventure Pumice
4,2022-10-23 03:00:00+00:00,1458.0,0.0,0.00,NaN,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1712,2023-03-02 07:45:02+00:00,994.0,127.0,12.78,TTD-221112-SP-V-S23MUF-BH73-BLACK,TTD-221112-SP-V-S23MUF-BH73-BLACK,"CapBoy Trading Co.,Ltd",1502.0,LEAN 5,May 5,NaN,NaN,NaN,NaN
1713,2023-03-02 07:48:01+00:00,48.0,50.0,104.17,DYC-US-230126-Blacktiph-690-Opt03,DYC-US-230126-Blacktiph-690-Opt03,DESIGN YOUR CAPS USA LLC,250.0,LEAN 1,May 1,NaN,NaN,NaN,NaN
1714,2023-03-02 07:51:02+00:00,994.0,83.0,8.35,CIE-221218-Distance Nirvana BK Hat SM,CIE-221218-Distance Nirvana BK Hat SM,Ciele Athletics Inc,289.0,LEAN 6,May 6,NaN,NaN,NaN,NaN
1715,2023-03-02 07:54:03+00:00,994.0,116.0,11.67,TTD-221112-SP-V-S23SUC-BH11-LK,TTD-221112-SP-V-S23SUC-BH11-LK,"CapBoy Trading Co.,Ltd",4000.0,LEAN 4,May 4,NaN,NaN,NaN,NaN


In [164]:
dfProductionTrackingFix = dfProductionTracking.drop(columns= {'id','updated_at', 'deleted_at' , 'order_id' , 'manufacture_dept_id', 'order_id', 'prod_not_exist_order_id' })

new_col1 = ['code_order' , 'name_order', 'name_cus' ,'created_at', 'target_quantity' ,  'actual_quantity', 'quantity' ,'complete_percent', 'code_dept', 'name_dept' , 
      'name_not_exist', 'not_exist_create', 'buyer_name', 'style_name' ]
   
dfProductionTrackingFix= dfProductionTrackingFix.reindex(columns=new_col1)
   
dfProductionTrackingFix.rename(columns = { 'code_order': 'Mã hàng', 'name_order': 'Tên ĐH', 'name_cus' : 'Tên KH','created_at' : 'Ngày sản xuất' ,'target_quantity' : 'Mục tiêu' , 'actual_quantity' : 'thực tế', 'quantity' : 'Số lượng ĐH',
                                             'complete_percent' : 'Tỉ lệ HT' , 'code_dept' : 'Mã chuyền' , 'name_dept' : 'Tên chuyền' , 'name_not_exist': 'ĐH khác' , 'not_exist_create': 'Ngày tạo'}, inplace = True)
dfProductionTrackingFix

,Mã hàng,Tên ĐH,Tên KH,Ngày sản xuất,Mục tiêu,thực tế,Số lượng ĐH,Tỉ lệ HT,Mã chuyền,Tên chuyền,ĐH khác,Ngày tạo,buyer_name,style_name
0,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,927.0,318.0,NaN,34.30,LEAN 3,May 3,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet
1,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,301.0,0.0,NaN,0.00,LEAN 4,May 4,PO38046SAND,2022-10-22 03:00:00+00:00,SA,PO 38046 Charter Hat Sand
2,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,268.0,2.0,NaN,0.75,LEAN 5,May 5,PO38050ROL_TAN,2022-10-22 03:00:00+00:00,SA,PO38050 Kids Clear Creek Boonie
3,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,289.0,0.0,NaN,0.00,LEAN 6,May 6,PO38055_ULTR_PUMI,2022-10-22 03:00:00+00:00,SA,PO38056_Ultra adventure Pumice
4,NaN,NaN,NaN,2022-10-23 03:00:00+00:00,1458.0,0.0,NaN,0.00,LEAN 3,May 3,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1712,TTD-221112-SP-V-S23MUF-BH73-BLACK,TTD-221112-SP-V-S23MUF-BH73-BLACK,"CapBoy Trading Co.,Ltd",2023-03-02 07:45:02+00:00,994.0,127.0,1502.0,12.78,LEAN 5,May 5,NaN,NaN,NaN,NaN
1713,DYC-US-230126-Blacktiph-690-Opt03,DYC-US-230126-Blacktiph-690-Opt03,DESIGN YOUR CAPS USA LLC,2023-03-02 07:48:01+00:00,48.0,50.0,250.0,104.17,LEAN 1,May 1,NaN,NaN,NaN,NaN
1714,CIE-221218-Distance Nirvana BK Hat SM,CIE-221218-Distance Nirvana BK Hat SM,Ciele Athletics Inc,2023-03-02 07:51:02+00:00,994.0,83.0,289.0,8.35,LEAN 6,May 6,NaN,NaN,NaN,NaN
1715,TTD-221112-SP-V-S23SUC-BH11-LK,TTD-221112-SP-V-S23SUC-BH11-LK,"CapBoy Trading Co.,Ltd",2023-03-02 07:54:03+00:00,994.0,116.0,4000.0,11.67,LEAN 4,May 4,NaN,NaN,NaN,NaN


In [165]:
dfProductionTrackingFix.to_excel('../dataDA/get_production_tracking.xlsx')


In [166]:
dfProductionTracking.head(5)

,id,created_at,updated_at,deleted_at,order_id,prod_not_exist_order_id,manufacture_dept_id,target_quantity,actual_quantity,complete_percent,code_order,name_order,name_cus,quantity,code_dept,name_dept,name_not_exist,not_exist_create,buyer_name,style_name
0,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.30,NaN,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet
1,2,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,2.0,8,301.0,0.0,0.00,NaN,NaN,NaN,NaN,LEAN 4,May 4,PO38046SAND,2022-10-22 03:00:00+00:00,SA,PO 38046 Charter Hat Sand
2,3,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,3.0,9,268.0,2.0,0.75,NaN,NaN,NaN,NaN,LEAN 5,May 5,PO38050ROL_TAN,2022-10-22 03:00:00+00:00,SA,PO38050 Kids Clear Creek Boonie
3,4,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,4.0,10,289.0,0.0,0.00,NaN,NaN,NaN,NaN,LEAN 6,May 6,PO38055_ULTR_PUMI,2022-10-22 03:00:00+00:00,SA,PO38056_Ultra adventure Pumice
4,5,2022-10-23 03:00:00+00:00,2022-12-15 10:01:28+00:00,NaN,NaN,1.0,7,1458.0,0.0,0.00,NaN,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet


In [167]:
dfWorkingTime = pd.read_csv('../data_csv/WorkingTime.csv')
dfProductionTrackingDetail = pd.read_csv('../data_csv/ProductionTrackingDetail.csv')
dfProductionTrackingDetail = dfProductionTrackingDetail.drop(columns= {'deleted_at' , 'updated_at'}).rename(columns = {'created_at' : 'created_detail' , 'quantity' : 'quantity_detail' ,
                                                                                                       'production_tracking_id' : 'id' ,'manufacture_dept_id' : 'manufacture_dept_id_detail' , 'target_quantity' : "target_quantity_detail" })

dfProductionTrackingDetail
dfWorkingTime = dfWorkingTime.rename(columns= {'id' : 'working_time_id' , 'code' : 'time_working'})
dfWorkingTime


,working_time_id,time_working
0,1,07:00-07:30
1,2,07:30-08:30
2,3,08:30-09:30
3,4,09:30-10:30
4,5,10:30-11:30
5,6,11:30-13:15
6,7,13:15-14:15
7,8,14:15-15:15
8,9,15:15-16:15
9,10,16:15-17:45


In [168]:
dfProdTrackingDetail = dfProductionTracking.merge(dfProductionTrackingDetail , on = ['id' , 'prod_not_exist_order_id' , 'order_id'] , how = 'right')
dfProdTrackingDetail = dfProdTrackingDetail.merge(dfWorkingTime , on = 'working_time_id' , how = 'left')
dfProdTrackingDetail

,id,created_at,updated_at,deleted_at,order_id,prod_not_exist_order_id,manufacture_dept_id,target_quantity,actual_quantity,complete_percent,...,name_not_exist,not_exist_create,buyer_name,style_name,quantity_detail,created_detail,working_time_id,manufacture_dept_id_detail,target_quantity_detail,time_working
0,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,1,7,0.0,07:00-07:30
1,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,2,7,0.0,07:30-08:30
2,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,3,7,0.0,08:30-09:30
3,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,4,7,0.0,09:30-10:30
4,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,5,7,0.0,10:30-11:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27467,1743,2023-03-02 09:30:02+00:00,2023-03-02 10:12:05+00:00,NaN,2876.0,NaN,5,2868.0,175.0,6.1,...,NaN,NaN,NaN,NaN,0.0,2023-03-02 09:30:02+00:00,12,5,0.0,18:45-19:45
27468,1743,2023-03-02 09:30:02+00:00,2023-03-02 10:12:05+00:00,NaN,2876.0,NaN,5,2868.0,175.0,6.1,...,NaN,NaN,NaN,NaN,0.0,2023-03-02 09:30:02+00:00,13,5,0.0,19:45-20:45
27469,1743,2023-03-02 09:30:02+00:00,2023-03-02 10:12:05+00:00,NaN,2876.0,NaN,5,2868.0,175.0,6.1,...,NaN,NaN,NaN,NaN,0.0,2023-03-02 09:30:02+00:00,14,5,0.0,20:45-21:45
27470,1743,2023-03-02 09:30:02+00:00,2023-03-02 10:12:05+00:00,NaN,2876.0,NaN,5,2868.0,175.0,6.1,...,NaN,NaN,NaN,NaN,0.0,2023-03-02 09:30:02+00:00,15,5,0.0,21:45-22:00


In [169]:
dfProdTrackingDetail[(dfProdTrackingDetail['name_not_exist'] == '1666975234DH1079') & (dfProdTrackingDetail['id'] == 1)]

,id,created_at,updated_at,deleted_at,order_id,prod_not_exist_order_id,manufacture_dept_id,target_quantity,actual_quantity,complete_percent,...,name_not_exist,not_exist_create,buyer_name,style_name,quantity_detail,created_detail,working_time_id,manufacture_dept_id_detail,target_quantity_detail,time_working
0,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,1,7,0.0,07:00-07:30
1,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,2,7,0.0,07:30-08:30
2,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,3,7,0.0,08:30-09:30
3,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,4,7,0.0,09:30-10:30
4,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,5,7,0.0,10:30-11:30
5,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,19.0,2022-10-22 03:00:00+00:00,6,7,19.0,11:30-13:15
6,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,73.0,2022-10-22 03:00:00+00:00,7,7,110.0,13:15-14:15
7,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,11.0,2022-10-22 03:00:00+00:00,8,7,110.0,14:15-15:15
8,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,215.0,2022-10-22 03:00:00+00:00,9,7,110.0,15:15-16:15
9,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,10,7,110.0,16:15-17:45


In [170]:
dfProdTrackingDetail.head(2)

,id,created_at,updated_at,deleted_at,order_id,prod_not_exist_order_id,manufacture_dept_id,target_quantity,actual_quantity,complete_percent,...,name_not_exist,not_exist_create,buyer_name,style_name,quantity_detail,created_detail,working_time_id,manufacture_dept_id_detail,target_quantity_detail,time_working
0,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,1,7,0.0,07:00-07:30
1,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,2,7,0.0,07:30-08:30


In [171]:
dfProdTrackingDetailFix = dfProdTrackingDetail.drop(columns= {'updated_at','id', 'deleted_at' , 'order_id' , 'prod_not_exist_order_id', 'manufacture_dept_id', 'working_time_id' ,
                                                              'manufacture_dept_id_detail' , 'not_exist_create'})

dfProdTrackingDetailFix

,created_at,target_quantity,actual_quantity,complete_percent,code_order,name_order,name_cus,quantity,code_dept,name_dept,name_not_exist,buyer_name,style_name,quantity_detail,created_detail,target_quantity_detail,time_working
0,2022-10-22 03:00:00+00:00,927.0,318.0,34.3,NaN,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,0.0,07:00-07:30
1,2022-10-22 03:00:00+00:00,927.0,318.0,34.3,NaN,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,0.0,07:30-08:30
2,2022-10-22 03:00:00+00:00,927.0,318.0,34.3,NaN,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,0.0,08:30-09:30
3,2022-10-22 03:00:00+00:00,927.0,318.0,34.3,NaN,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,0.0,09:30-10:30
4,2022-10-22 03:00:00+00:00,927.0,318.0,34.3,NaN,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,0.0,10:30-11:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27467,2023-03-02 09:30:02+00:00,2868.0,175.0,6.1,DYC-US-230126-Blacktiph-690-Opt12,DYC-US-230126-Blacktiph-690-Opt12,DESIGN YOUR CAPS USA LLC,250.0,LEAN 1,May 1,NaN,NaN,NaN,0.0,2023-03-02 09:30:02+00:00,0.0,18:45-19:45
27468,2023-03-02 09:30:02+00:00,2868.0,175.0,6.1,DYC-US-230126-Blacktiph-690-Opt12,DYC-US-230126-Blacktiph-690-Opt12,DESIGN YOUR CAPS USA LLC,250.0,LEAN 1,May 1,NaN,NaN,NaN,0.0,2023-03-02 09:30:02+00:00,0.0,19:45-20:45
27469,2023-03-02 09:30:02+00:00,2868.0,175.0,6.1,DYC-US-230126-Blacktiph-690-Opt12,DYC-US-230126-Blacktiph-690-Opt12,DESIGN YOUR CAPS USA LLC,250.0,LEAN 1,May 1,NaN,NaN,NaN,0.0,2023-03-02 09:30:02+00:00,0.0,20:45-21:45
27470,2023-03-02 09:30:02+00:00,2868.0,175.0,6.1,DYC-US-230126-Blacktiph-690-Opt12,DYC-US-230126-Blacktiph-690-Opt12,DESIGN YOUR CAPS USA LLC,250.0,LEAN 1,May 1,NaN,NaN,NaN,0.0,2023-03-02 09:30:02+00:00,0.0,21:45-22:00


In [172]:
dfProdTrackingDetailFix = dfProdTrackingDetail.drop(columns= {'updated_at','id', 'deleted_at' , 'order_id' , 'prod_not_exist_order_id', 'manufacture_dept_id', 'working_time_id' ,
                                                              'manufacture_dept_id_detail' , 'not_exist_create' , 'style_name'})

new_col1 = ['code_dept' , 'name_dept' , 'code_order' , 'name_order' , 'name_cus' ,'created_at', 'time_working' ,'target_quantity_detail' , 'quantity_detail' , 'target_quantity', 'actual_quantity', 'complete_percent' ,  'quantity' ,
      'name_not_exist', 'not_exist_create', 'buyer_name', 'style_name' ]
   
dfProdTrackingDetailFix= dfProdTrackingDetailFix.reindex(columns=new_col1)
   
dfProdTrackingDetailFix.rename(columns = { 'code_dept': 'Mã chuyền', 'name_dept': 'Tên chuyền', 'code_order' : 'Mã ĐH' ,'name_order' : 'Tên ĐH', 'name_not_exist' : 'Tên ĐH không tồn tại' , 'name_cus' : 'Tên Khách hàng', 'buyer_name' : 'Tên Khách hàng khác',
                                             'created_at' : 'Ngày SX' , 'not_exist_create' : 'Ngày SX đơn hàng khác' , 'time_working' : 'Thời gian SX' , 'target_quantity_detail' : 'SL mục tiêu theo mốc TG' , 'quantity_detail': 'SL theo mốc TG' , 'target_quantity': 'SL mục tiêu tổng' , 
                                             'actual_quantity' : 'SL thực tế' , 'complete_percent' : 'Tỉ lệ hoàn thành' , 'quantity' : 'Số lượng ĐH'}, inplace = True)
dfProdTrackingDetailFix

,Mã chuyền,Tên chuyền,Mã ĐH,Tên ĐH,Tên Khách hàng,Ngày SX,Thời gian SX,SL mục tiêu theo mốc TG,SL theo mốc TG,SL mục tiêu tổng,SL thực tế,Tỉ lệ hoàn thành,Số lượng ĐH,Tên ĐH không tồn tại,Ngày SX đơn hàng khác,Tên Khách hàng khác,style_name
0,LEAN 3,May 3,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,07:00-07:30,0.0,0.0,927.0,318.0,34.3,NaN,1666975234DH1079,NaN,PO 38056 GODDESS,NaN
1,LEAN 3,May 3,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,07:30-08:30,0.0,0.0,927.0,318.0,34.3,NaN,1666975234DH1079,NaN,PO 38056 GODDESS,NaN
2,LEAN 3,May 3,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,08:30-09:30,0.0,0.0,927.0,318.0,34.3,NaN,1666975234DH1079,NaN,PO 38056 GODDESS,NaN
3,LEAN 3,May 3,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,09:30-10:30,0.0,0.0,927.0,318.0,34.3,NaN,1666975234DH1079,NaN,PO 38056 GODDESS,NaN
4,LEAN 3,May 3,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,10:30-11:30,0.0,0.0,927.0,318.0,34.3,NaN,1666975234DH1079,NaN,PO 38056 GODDESS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27467,LEAN 1,May 1,DYC-US-230126-Blacktiph-690-Opt12,DYC-US-230126-Blacktiph-690-Opt12,DESIGN YOUR CAPS USA LLC,2023-03-02 09:30:02+00:00,18:45-19:45,0.0,0.0,2868.0,175.0,6.1,250.0,NaN,NaN,NaN,NaN
27468,LEAN 1,May 1,DYC-US-230126-Blacktiph-690-Opt12,DYC-US-230126-Blacktiph-690-Opt12,DESIGN YOUR CAPS USA LLC,2023-03-02 09:30:02+00:00,19:45-20:45,0.0,0.0,2868.0,175.0,6.1,250.0,NaN,NaN,NaN,NaN
27469,LEAN 1,May 1,DYC-US-230126-Blacktiph-690-Opt12,DYC-US-230126-Blacktiph-690-Opt12,DESIGN YOUR CAPS USA LLC,2023-03-02 09:30:02+00:00,20:45-21:45,0.0,0.0,2868.0,175.0,6.1,250.0,NaN,NaN,NaN,NaN
27470,LEAN 1,May 1,DYC-US-230126-Blacktiph-690-Opt12,DYC-US-230126-Blacktiph-690-Opt12,DESIGN YOUR CAPS USA LLC,2023-03-02 09:30:02+00:00,21:45-22:00,0.0,0.0,2868.0,175.0,6.1,250.0,NaN,NaN,NaN,NaN


In [173]:
dfProdTrackingDetailFix = dfProdTrackingDetailFix.sort_values("Ngày SX" , ascending=False) 
dfProductionTrackingFix = dfProductionTrackingFix.sort_values("Ngày sản xuất" , ascending=False) 


In [174]:
with pd.ExcelWriter('../dataDA/get_production_tracking.xlsx') as writer:  
    dfProdTrackingDetailFix.to_excel(writer, sheet_name='tracking_detail')
    dfProductionTrackingFix.to_excel(writer, sheet_name='tracking')
